In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.nn import Parameter
from torch.autograd import Variable

from math import pi, log
from BBSVI import SVI

# Gaussian Mixture Example

Assume we have mixture of some Gaussians and we want to distinguish them. Here we do it using Black Box SVI.

In [2]:
# generating data

std = 1
mu = np.array([-5, 5])
num_components = 2

np.random.seed(42)
num_samples = 100

components = np.random.randint(num_components, size=num_samples)
data = torch.Tensor(np.random.normal(mu[components], std))

Below we define two classes for prior and variational distributions:

In [3]:
class GaussianMixture:
    def __init__(self, num_components, std=1.):
        self.std = std
        self.num_components = num_components
        self.prior = torch.distributions.Normal(0, std)
        
    def log_likelihood_global(self, beta):
        return torch.sum(self.prior.log_prob(beta))
    
    def log_likelihood_local(self, z, beta):
        return torch.ones(1) / self.num_components
    
    def log_likelihood_joint(self, x, z, beta):
        dist = torch.distributions.Normal(beta[z], 1)
        return dist.log_prob(x)

In [4]:
class VariationalDistribution:
    def __init__(self, num_components, data_size):
        self.num_components = num_components
        self.means = torch.nn.Parameter(torch.normal(torch.zeros(num_components), torch.ones(num_components)))
        self.log_std = torch.nn.Parameter(torch.zeros(num_components))
        self.probs = torch.nn.Parameter(torch.ones(data_size, num_components) / num_components)
        self.parameters = [self.means, self.log_std, self.probs]
        
    def sample_global(self):
        return torch.normal(self.means, torch.exp(self.log_std))
    
    def sample_local(self, beta, idx):
        probs = self.probs[idx].data.numpy()
        z = np.random.choice(self.num_components, p=probs / np.sum(probs))
        return torch.LongTensor([z])
    
    def log_likelihood_global(self, beta):
        return torch.sum((beta - self.means) ** 2 / (2 * torch.exp(2 * self.log_std)) - self.log_std - log(2 * pi) / 2)
    
    def log_likelihood_local(self, z, idx):
        return self.probs[idx, z] / torch.sum(self.probs[idx])

Define prior, variational distribution and optimizer

In [5]:
prior = GaussianMixture(num_components, std=10)
var = VariationalDistribution(num_components, num_samples)

opt = torch.optim.Adam(var.parameters, lr=1e-3)

Do the inference

In [6]:
svi = SVI(data, prior, var, opt)

In [7]:
svi.make_inference(num_steps=1000, shuffle=False, print_progress=True)

........................

Results:

In [8]:
predicted_components = torch.max(var.probs, dim=1)[1].data.numpy()
accuracy = np.sum(predicted_components == components) / len(predicted_components)
accuracy = max(accuracy, 1 - accuracy)
print('Mixture components detecting accuracy: %.2f %%' % (accuracy * 100))

predicted_mu = var.means.data.numpy()
print('Predicted means: \t %.2f \t %.2f' % (predicted_mu[0], predicted_mu[1]))
print('Real means:      \t %.2f \t %.2f' % (mu[0], mu[1]))

Mixture components detecting accuracy: 100.00 %
Predicted means: 	 -1.33 	 -0.42
Real means:      	 -5.00 	 5.00


**NOTE:** SVI strongly depends on the initialization configuration. If you fail to reproduce good result, try to reinitialize initial parameters